# Recipe NER tagger

I shall build a custom NER tagger using SpaCy, trained on an annotated dataset of 1300 posts scraped from recipe hashtags on Instagram.

In [13]:
import pandas as pd
import json
import random

In [2]:
posts = pd.read_csv("/Users/maxkirwan/Desktop/Uni/Data Science MSc/Data Science Project/nutrition-insta/Instagram Data Scraping/Phantom Buster/recipe_posts.csv")
postst

postUrl  \
0     https://www.instagram.com/p/CgWcU2fss5n/   
1     https://www.instagram.com/p/CgckMjTpSPs/   
2     https://www.instagram.com/p/CgZqYK3q_t7/   
3     https://www.instagram.com/p/Cgdy9wBjeYl/   
4     https://www.instagram.com/p/Cgdp-q1vao_/   
...                                        ...   
1266  https://www.instagram.com/p/CgcTdXLI0Dd/   
1267  https://www.instagram.com/p/CgcS-dFJRmx/   
1268  https://www.instagram.com/p/CgcSs3GjRSE/   
1269  https://www.instagram.com/p/CgcStMdv2vK/   
1270  https://www.instagram.com/p/CgcSuw8oVL2/   

                                             profileUrl  \
0                  https://www.instagram.com/jadesbites   
1               https://www.instagram.com/sweettreatsyt   
2           https://www.instagram.com/gourmandisedezaza   
3                https://www.instagram.com/nicolejcooks   
4             https://www.instagram.com/its_shreyajoshi   
...                                                 ...   
1266  https://www.instagram.com/liz_sw_weightloss_jo...   
1267             https://www.instagram.com/_nanny_gram_   
1268          https://www.instagram.com/goodiesfoodhall   
1269           https://www.instagram.com/smoothie.smart   
1270         https://www.instagram.com/keto.diet.pro___   

                       username                       fullName  commentCount  \
0                    jadesbites          JADE | Recipes & Food            47   
1                 sweettreatsyt             Ania | SweetTreats             4   
2             gourmandisedezaza                   Zahramorrhad            85   
3                  nicolejcooks                    Nicole Jain             4   
4               its_shreyajoshi                   Food Blogger             7   
...                         ...                            ...           ...   
1266  liz_sw_weightloss_journey                            NaN             1   
1267               _nanny_gram_                        Felicia             0   
1268            goodiesfoodhall              Goodies Food Hall             2   
1269             smoothie.smart  Smoothies | Weight Los | Diet             3   
1270           keto.diet.pro___            KETO DIET PLANNER 🔥             4   

      likeCount                   pubDate  \
0          1648  2022-07-23T10:00:53.000Z   
1             3  2022-07-25T19:05:06.000Z   
2          1569  2022-07-24T16:01:24.000Z   
3            46  2022-07-26T06:33:24.000Z   
4             3  2022-07-26T05:14:53.000Z   
...         ...                       ...   
1266         13  2022-07-25T16:38:51.000Z   
1267          3  2022-07-25T16:34:38.000Z   
1268         18  2022-07-25T16:32:14.000Z   
1269          3  2022-07-25T16:32:16.000Z   
1270         36  2022-07-25T16:32:29.000Z   

                                            description          location  \
0     🧄 ~ A G L I O ~ E ~ O L I O ~ 🧄\n\nSimply mean...         Liverpool   
1     NEW! Condensed Milk Brownies. These brownies a...  Toronto, Ontario   
2     Brioche feuilletée de @jimmy_mornet que l’on r...               NaN   
3     HOT & COLD\n.\nRoasted peppers & tomatoes pair...               NaN   
4     Chocolate Bar\n\n#chocolate #recipes #recipeof...               NaN   
...                                                 ...               ...   
1266  Fancied burger pasta tonight but without the p...               NaN   
1267  Nothing better than starting a Monday with one...               NaN   
1268  Happy summer holidays! 🌞🎉 We hope you survived...     Pulham Market   
1269  AMAZING tranformation results from a customer ...               NaN   
1270  Double tap if you love it.\n\nFor recipe just ...               USA   

                                                 imgUrl               postId  \
0     https://scontent-lhr8-2.cdninstagram.com/v/t51...  2888620789210467943   
1     https://scontent-lhr8-1.cdninstagram.com/v/t51...  2890344253083689964   
2     https://scontent-lhr8-2.cdninstagram.com/v/t51...  28895

In [ ]:
# Saving posts as csv
posts.description.to_csv('recipe_descriptions.csv', index=False)

## Opening annotations file
Now that we have used LightTag to annotate the text data, we can use the results.

In [3]:
# Opening json file with annotations
f = open("entity-tagging_annotations.json")
annotations = json.load(f)

In [4]:
annotations

{'id': '1184d5bff9964bfeb36355ac2d5049f1',
 'examples': [{'content': 'need a little lunch inspo?\nhere you go!\n\na quick and easy omelette recipe that will satisfy 🤤 \n\nSAVE & ENJOY 🤌🏼\n\n#nutritionist\n#lunch\n#easyrecipes\n#dietitian\n#healthyeating',
   'seen_by': [],
   'metadata': {},
   'example_id': '4e10060f-9ffe-4d8d-9a87-555b41a8662c',
   'annotations': [],
   'classifications': []},
  {'content': 'Palačinke sa čokoladnim preljevom🤤\n\nSmjesa za palačinke:\nJaje, whey, mlijeko/voda, brašno\n\nKao nadjev za palačinke sam koristila proteinski puding od čokolade\n\nČokoladni preljev:\nKikiriki maslac, nutella, voda/mlijeko\n\nDobar tek!❤️\n\n#breakfastideas #healthyfood #healthyrecipes #loveyourself #foodphotography #foodporn #foodbalance #food #fitcooking #snack #snackideas #recipeideas #fitrecipes #recipes #volimkuhati #recepti #zdravirecepti #healthymeals #healthymealprep #healthycreations #mealprep #recepti #zdravirecepti #pancakes #healthypancakes #protein #highprotein #p

### Format for training data

The data for training the custom NER tagger must be of the form:

    TRAIN_DATA = [('3 tablespoons chopped fresh sage',
                {'entities': [(0, 1, 'QUANTITY), (2, 13, 'MEASUREMENT'), (28, 32, 'INGREDIENT')]}),
                ('1/4 cup brown sugar',
                {'entities': [(0, 3, 'QUANTITY), (4, 7, 'MEASUREMENT'), (8, 19, 'INGREDIENT')]}),
                ...
                ]

In [5]:
def load_json_to_df(file):
    
    results = json.load(open(file))
    
    # Create list of only those posts which have been annotated
    annotated = []
    for example in results['examples']:
        if example['annotations'] != []:
            annotated.append(example)
            
    return pd.DataFrame(annotated)


def convert_to_spacy_format(df):
    
    TRAIN_DATA = []
    
    for index, row in df.iterrows():
        
        sentence = row['content']
        annotations_input_list = row['annotations']
        
        annotations_output_list = []
        ing_dict = {}
        
        for annotation in annotations_input_list:
            
            annotations_output_list.append((annotation['start'], annotation['end'], annotation['tag']))
        
        ing_dict['entities'] = annotations_output_list
        TRAIN_DATA.append((sentence, ing_dict))
        
    return TRAIN_DATA

In [19]:
TRAIN_DATA = convert_to_spacy_format(load_json_to_df("entity-tagging_annotations.json"))

In [21]:
# Splitting into train and dev set
random.shuffle(TRAIN_DATA)
split_integer = int(len(TRAIN_DATA)*0.8)
train_set = TRAIN_DATA[:split_integer]
dev_set = TRAIN_DATA[split_integer:]

## SpaCy custom NER tagger

In [25]:
# Converting TRAIN_DATA to docbin spacy file

import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

def convert_to_spacy_docbin(dataset, file_location):
    
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(dataset):
        doc = nlp.make_doc(text) 
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="expand")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        print(ents)
        try:
            doc.ents = ents
        except:
            print(f"Error with document")
        db.add(doc)

    db.to_disk(file_location) # save the docbin object


convert_to_spacy_docbin(train_set, "./train.spacy")
convert_to_spacy_docbin(dev_set, "./dev.spacy")

100%|██████████████████████████████████████████| 26/26 [00:00<00:00, 126.36it/s]

[garlic, water, butter, salt, pepper, oil, 2, butter, 1, garlic, water, kg, paprika, 1, GARLIC, potatoes, dill]
[2, g, vanilla extract, 1tsp, 1tsp, 40, baking powder, g, eggs, oats, 1tsp, 175, yog, 1tsp]
Error with document
[cucumber, mayo]
[4, 11/2, oats milk, cacao powder, tablespoon, tablespoons, oats cream, 1, 2, bananas]
[oats]
[honey, balsamic glaze, basil, 作り方はバルサミコ酢1カップと砂糖(無しでも良い)大さじ1を弱火で15分ほど煮詰める, mascarpone cheese, olive oil, strawberries, olive oil, 作り方はバルサミコ酢1カップと砂糖(無しでも良い)大さじ1を弱火で15分ほど煮詰める]
Error with document
[basil, handful, 1, 2, pesto, 1, balsamic glaze, tomatoes, mozarella cheese, chicken breast]
[cucumber, Cucumber]
[1/2, 1/2, 1/2, mayo, lemon, teaspoon, teaspoon, turmeric, salt, cumin, cup, cumin, 1, 1/2, black pepper, paprika, 1/2, teaspoon, Half, olive oil, teaspoon, 1/2, teaspoon, chicken, 1/2, Pinch, chilli flakes, 1/2, tablespoon, chicken stock, teaspoon, Chicken, 2, Pinch, 1, salt, teaspoon, 1, Basmati rice, 1, 1/2, 1/2, 1, 1/2, parsley, garlic, teaspoon, pepp


100%|█████████████████████████████████████████████| 7/7 [00:00<00:00, 98.47it/s]

[6, black pepper, cucumber, 2, oz, bread, half, cream cheese, salt, TB, dill, 4, slices, lemon, butter, 1/2]
[1]
[mint, red onion, garlic]
[mint, 100, olive oil, pepper, onion, 1/2, olive oil, thyme, 1, 1/4, teaspoon, cup, gram, tablespoons, 1, 3, tablespoon, 1/4, sumac, cup, 1, garlic, parsley, chili, salt, red onion, teaspoon, 1/4, yogurt, tablespoons, 1, bread, 1/2, 1/2, eggplant, 2, tablespoon, cup, lime juice, cup, halloumi, 1, chili flakes, teaspoon, 1/2, 1, teaspoon, 1, 2, salt, dill, cloves, Pita bread]
Error with document
[1.550]
[Garlic, Lemon]
[1, 1, 2, 1️⃣, 2️⃣]


## The Problem...

The problem here is that, for some texts, the SpaCy model is not able to resolve all the entities being passed into it. Therefore an error results when we try to match the entities: `doc.ents = ents`

This is caused due to an issue with the annotations, where multiple annotated spans relate to the same token (or tokens). There are two reasons for this:

1. Multiple spans relate to the same token (A MISTAKE!), e.g. `tomato ketchup` has two spans: `(0, 6, 'INGREDIENT')` and `(0, 14, 'INGREDIENT')`. This is a mistake in annotating and should be rectified in the annotations.

2. Multiple spans relate to the same token (NOT A MISTAKE!!), e.g. a problem arises in `25ml chicken stock`. SpaCy converts `25ml` into one token, but two seperate annotations relate to this token: `(0, 2, 'QUANTITY)` and `(2, 4, 'MEASUREMENT)`. This is a more difficult problem, which may involve adjusting SpaCy's tokenisation to resolve.

## Custom NER tagger
Let's exclude these problematic scenarios and train our custom NER tagger without them.

In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./dev.spacy

In [ ]:
nlp_ner = spacy.load("./model-best")

In [ ]:
doc = nlp_ner(posts.description[926])

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)